| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | X-meshgrid |
| Y | array (Ny, Nx) | Y-meshgrid |
| C | array (Ny, Nx, 2, 2) | Cauchy-Green strain tensor over meshgrid |
| x_vortex | float | x-coordinate of vortex center |
| y_vortex | float | y-coordinate of vortex center |
| len_poincare | float | length of poincare map |
| step_size | float | step size used for integration of shearline |
| incompressible | bool | Incompressible/Compressible |

In [3]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])

# add Algorithm folder to current working path
sys.path.append(parent_directory+"/General_Functions")

In [4]:
# Import function which calculates eigenvalues/eigenvectors in 2D
from ipynb.fs.defs.loc_max import _loc_max
from ipynb.fs.defs.orient_vector_field import _orient_vector_field
from ipynb.fs.defs.RK4_tensorlines import _RK4_tensorlines
from ipynb.fs.defs.eigen import eigen

from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

def _shearlines(X, Y, C, lamda, x_vortex, y_vortex, len_poincare, length_threshold, step_size, incompressible, sign_eta):
    
    lambda_1 = np.zeros((C.shape[0], C.shape[1]))*np.nan
    lambda_2 = np.zeros((C.shape[0], C.shape[1]))*np.nan
    v1 = np.zeros((C.shape[0], C.shape[1], 2))
    v2 = np.zeros((C.shape[0], C.shape[1], 2))
        
    # vector_field
    vector_field = np.zeros((C.shape[0], C.shape[1], 2))
    
    for i in range(lambda_1.shape[0]):
        for j in range(lambda_2.shape[1]):
            lambda_1[i, j], lambda_2[i, j], v1[i,j,:], v2[i,j,:] = eigen(C[i, j, :, :], incompressible)
            if np.isfinite(lambda_1[i,j]):
                denominator = lambda_2[i, j] - lambda_1[i, j]
                if lambda_2[i,j]-lamda**2 >= 0 and lamda**2-lambda_1[i,j]>=0:
                    vector_field[i,j,:] = np.sqrt((lambda_2[i,j]-lamda**2)/denominator)*v1[i,j,:]+sign_eta*np.sqrt((lamda**2-lambda_1[i,j])/denominator)*v2[i,j,:]
    
    # Defined domain
    defined_domain = np.isfinite(lambda_1).astype(int)
    
    # Set lambda_1/lambda_2 values to zero if they are 'nan'
    # This step is necessary for the gridded interpolation
    for i in range(lambda_1.shape[0]):
        for j in range(lambda_1.shape[1]):
            if np.isnan(lambda_1[i, j]):
                lambda_1[i, j] = 0
                lambda_2[i, j] = 0
    
    # Import Rectangular bivariate spline from scipy
    from scipy.interpolate import RectBivariateSpline as RBS
        
    # Interpolate Eigenvalue field
    interp_lambda_1 = RBS(Y[:,0], X[0,:], lambda_1, kx = 1, ky = 1)
    interp_lambda_2 = RBS(Y[:,0], X[0,:], lambda_2, kx = 1, ky = 1)
    
    # Define Poincare-section
    if x_vortex + len_poincare >= X[0,-2]:
        
        len_poincare = X[0,-2] - x_vortex
        
    x_pc = x_vortex + np.linspace(0, len_poincare, 100)
    y_pc = y_vortex + np.linspace(0, 0, 100)
    
    # Define list of shearlines
    shearlines = [[], []]
    
    # Distance between intersection of poincare section with shearline and 
    # initial point from where shearline is computed on poincare section
    distance_ps_s = []
    
    # Iterate over points of poincare-section
    for i in tqdm(range(len(x_pc))):
        
        # Boolean intersection of shearline with poincare section
        bool_intersection = False
        
        shearline = [[], []]
        
        # Local maxima point
        x = np.array([x_pc[i], y_pc[i]])
        
        # Boolean integration
        bool_integration = True
                
        # Check orientation of vector-field and rieorient if needed.
        vx, vy = _orient_vector_field(X, Y, x, vector_field) 
            
        # Initial vector orientation
        x_prime = np.array([vx, vy])
        
        if vy > 0:
            intersection_type = "neg_to_pos"
        else:
            intersection_type = "pos_to_neg"
            
        # Initial length
        length = 0
        
        # Append point to shearline
        for ii in range(2):
            shearline[ii].append(x[ii])
            
        # Start Integration with dummy variable 's_array'
        while bool_integration:
                    
            # RK4 integration for tensorline
            x, x_prime = _RK4_tensorlines(X, Y, defined_domain, x, x_prime, step_size, vector_field, interp_lambda_2, interp_lambda_1) 
            
            # If 'x' is defined then update length
            if x is not None:
                  
                # If x_prime is close to zero --> then we are close to a singularity of C
                # --> stop Integration
                
                if np.sqrt(x_prime[0]**2+x_prime[1]**2) < 10**(-6):
                    bool_integration = False
        
                # Compute length of shearline
                length += np.sqrt(x_prime[0]**2+x_prime[1]**2)*step_size
                
                # If length of shearline exceeds threshold
                # --> Stop integration
                if length > length_threshold:
            
                    bool_integration = False
        
                x1 = [shearline[0][-1], shearline[1][-1]]
                x2 = [x[0], x[1]]
                x3 = [x_pc[0], y_pc[0]]
                x4 = [x_pc[-1], y_pc[-1]]
                
                if intersection_type == "neg_to_pos":
                    Condition_intersection = x1[1] < x3[1] < x2[1]
                elif intersection_type == "pos_to_neg":
                    Condition_intersection = x2[1] < x3[1] < x1[1]
                    
                if x3[0] <= x1[0] <= x4[0] and x3[0] < x2[0] < x4[0] and Condition_intersection:
                    
                    x_intersec = compute_intersection(x1, x2, x3, x4)
                
                    for j in range(2):
                        shearline[j].append(x_intersec[j])
                     
                    # compute distance between intersection of shearline with poincare-section 
                    # and initial point of shearline
                    distance_ps_s.append(shearline[0][-1]-shearline[0][0])
                    
                    for j in range(2):
                        shearlines[j].append(shearline[j])
                    
                    # Integration will stop!!!
                    bool_integration = False
                    
                # If there is no intersection --> Append point to shearline and continue integration 
                else:
                    for j in range(2):
                        shearline[j].append(x[j])
                    
            else:
                        
                bool_integration = False
    
        #if bool_intersection and np.max(distance_ps_s)>0 and np.min(distance_ps_s) <0:
        #    for j in range(2):
        #        shearlines[j].append(shearline[j])
        
    return shearlines, distance_ps_s, vector_field

In [8]:
def compute_intersection(x1, x2, x3, x4):
    px= ((x1[0]*x2[1]-x1[1]*x2[0])*(x3[0]-x4[0])-(x1[0]-x2[0])*(x3[0]*x4[1]-x3[1]*x4[0]))/((x1[0]-x2[0])*(x3[1]-x4[1])-(x1[1]-x2[1])*(x3[0]-x4[0]) ) 
    py= ((x1[0]*x2[1]-x1[1]*x2[0])*(x3[1]-x4[1])-(x1[1]-x2[1])*(x3[0]*x4[1]-x3[1]*x4[0]))/((x1[0]-x2[0])*(x3[1]-x4[1])-(x1[1]-x2[1])*(x3[0]-x4[0]) )
    return px, py